In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trackml-particle-identification/train_sample.zip
/kaggle/input/trackml-particle-identification/train_3.zip
/kaggle/input/trackml-particle-identification/train_2.zip
/kaggle/input/trackml-particle-identification/sample_submission.csv.zip
/kaggle/input/trackml-particle-identification/train_5.zip
/kaggle/input/trackml-particle-identification/detectors.zip
/kaggle/input/trackml-particle-identification/test.zip
/kaggle/input/trackml-particle-identification/blacklist_training.zip
/kaggle/input/trackml-particle-identification/train_4.zip
/kaggle/input/trackml-particle-identification/train_1.zip


In [2]:
!pip install torch_geometric torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-2.8.0+cu126.html


Looking in links: https://data.pyg.org/whl/torch-2.8.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 66.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 111.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.2 MB/s eta 0:00:00a 0:00:01


In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset
from torch_geometric.transforms import KNNGraph
from torch_geometric.loader import DataLoader
from torch_geometric.nn import TransformerConv
import numpy as np
import os
import pandas as pd
import pytorch_lightning as pl
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
import zipfile
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import Counter
import gc

In [4]:
torch.cuda.empty_cache()
gc.collect()
print(" GPU memory cleared")

 GPU memory cleared


In [5]:

# Check available GPU memory
if torch.cuda.is_available():
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"Total GPU memory: {total_memory:.2f} GB")
    print(f"Available GPU memory: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB allocated")


Total GPU memory: 14.74 GB
Available GPU memory: 0.00 GB allocated


In [6]:
# Data exploration utility
def explore_data_files(root):
    """Explore the structure of data files"""

    print("DATA FILE EXPLORATION")

    
    data_files = {
        'train_sample': 'train_sample.zip',
        'train_1': 'train_1.zip',
        'train_2': 'train_2.zip',
        'train_3': 'train_3.zip',
        'train_4': 'train_4.zip',
        'train_5': 'train_5.zip',
        'test': 'test.zip',
    }
    
    for name, filename in data_files.items():
        filepath = os.path.join(root, filename)
        if os.path.exists(filepath):
            with zipfile.ZipFile(filepath, 'r') as z:
                files = z.namelist()
                events = len([f for f in files if f.endswith('-hits.csv')])
                print(f"\n{name:15} ({filename:20}): {events:4} events")
        else:
            print(f"\n{name:15} ({filename:20}): NOT FOUND")
    

In [7]:
# Explore data before training
root = '/kaggle/input/trackml-particle-identification'
explore_data_files(root)

DATA FILE EXPLORATION

train_sample    (train_sample.zip    ):  100 events

train_1         (train_1.zip         ): 1770 events

train_2         (train_2.zip         ): 1770 events

train_3         (train_3.zip         ): 1770 events

train_4         (train_4.zip         ): 1770 events

train_5         (train_5.zip         ): 1770 events

test            (test.zip            ):  125 events


In [8]:
# Manual data loading from zip
def load_event(zip_path, inner_path):
    with zipfile.ZipFile(zip_path, 'r') as z:
        hits = pd.read_csv(z.open(f'{inner_path}-hits.csv'))
        cells = pd.read_csv(z.open(f'{inner_path}-cells.csv'))
        particles = pd.read_csv(z.open(f'{inner_path}-particles.csv'))
        truth = pd.read_csv(z.open(f'{inner_path}-truth.csv'))
    return hits, cells, particles, truth


In [10]:
# Dataset class adjusted for Kaggle zip structure with memory-efficient sampling
class TrackMLDataset(Dataset):
    def __init__(self, root, zip_name='train_sample.zip', n_events=8, transform=None, max_hits=30000):
        super().__init__(root, transform=transform)
        self.zip_path = os.path.join(root, zip_name)
        self.max_hits = max_hits  # Configurable hit limit
        # Reduce k from 8 to 6 to save memory
        self.transform = transform or KNNGraph(k=6)
        
        # Dynamically extract unique event paths from zip namelist
        self.inner_paths = []
        with zipfile.ZipFile(self.zip_path, 'r') as z:
            for fname in z.namelist():
                if fname.endswith('-hits.csv'):
                    inner = fname.rsplit('-hits.csv', 1)[0]
                    self.inner_paths.append(inner)
        self.inner_paths = sorted(set(self.inner_paths))[:n_events]

    def len(self):
        return len(self.inner_paths)

    def get(self, idx):
        inner_path = self.inner_paths[idx]
        hits, cells, particles, truth = load_event(self.zip_path, inner_path)
        
        # MEMORY OPTIMIZATION: Sample hits if too many
        if len(hits) > self.max_hits:
            sample_idx = np.random.choice(len(hits), self.max_hits, replace=False)
            sample_idx = np.sort(sample_idx)  # Keep order for consistency
            hits = hits.iloc[sample_idx].reset_index(drop=True)
            # Update truth to match sampled hits
            truth = truth[truth['hit_id'].isin(hits['hit_id'])].reset_index(drop=True)
        
        # Node features (r, phi, z)
        r = np.sqrt(hits['x']**2 + hits['y']**2).values
        phi = np.arctan2(hits['y'], hits['x']).values
        z = hits['z'].values
        x = torch.tensor(np.stack([r, phi, z], axis=1), dtype=torch.float)
        
        # Store global coordinates for KNN calculation
        pos = torch.tensor(hits[['x', 'y', 'z']].values, dtype=torch.float)
        
        data = Data(x=x, pos=pos)
        
        # Apply KNN Transform to create edges
        if self.transform:
            data = self.transform(data)
            
            # Label the edges: 1 if both hits belong to the same particle, else 0
            hit_to_particle = truth.set_index('hit_id')['particle_id'].to_dict()
            
            raw_hit_ids = hits['hit_id'].values
            edge_src_pids = np.array([hit_to_particle.get(raw_hit_ids[i], 0) for i in data.edge_index[0]])
            edge_tgt_pids = np.array([hit_to_particle.get(raw_hit_ids[i], 0) for i in data.edge_index[1]])
            
            # Logic: same particle AND not noise (PID 0 is noise)
            y = (edge_src_pids == edge_tgt_pids) & (edge_src_pids != 0)
            data.y = torch.tensor(y, dtype=torch.float)
            
        return data

In [11]:
import torch.nn.functional as F

class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=0.45, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        # Use built-in stable BCE
        p = torch.sigmoid(inputs)
        # Add epsilon (1e-7) to prevent log(0) and numerical underflow in 16-bit
        ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
        p_t = p * targets + (1 - p) * (1 - targets)
        loss = ce_loss * ((1 - p_t) ** self.gamma)

        if self.alpha >= 0:
            alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
            loss = alpha_t * loss

        return loss.mean()

In [12]:
class TrackLightning(pl.LightningModule):
    def __init__(self, model, pos_threshold=0.35):
        super().__init__()
        self.model = model
        self.pos_threshold = pos_threshold
        
        self.loss_fn = FocalLoss(alpha=0.45, gamma=2.0)
        
        self.validation_step_outputs = []

    def forward(self, batch):
        return self.model(batch.x, batch.edge_index, batch.batch)

    def training_step(self, batch, batch_idx):
        logits = self(batch)
        logits = torch.clamp(logits, min=-10, max=10) # Numerical stability
        
        loss = self.loss_fn(logits, batch.y.float())
        
        if torch.isnan(loss):
            return None # Skip bad batches
            
        with torch.no_grad():
            # Lower threshold helps catch more tracks in early training
            preds = torch.sigmoid(logits) > self.pos_threshold
            acc = (preds == batch.y).float().mean()
        
        self.log('train_loss', loss, prog_bar=True, batch_size=batch.num_graphs)
        self.log('train_acc', acc, prog_bar=True, batch_size=batch.num_graphs)
        return loss

    def validation_step(self, batch, batch_idx):
        logits = self(batch)
        loss = self.loss_fn(logits, batch.y.float())
        
        # Lower threshold for validation too
        preds = (torch.sigmoid(logits) > self.pos_threshold).cpu()
        targets = batch.y.cpu()
        
        self.validation_step_outputs.append({'preds': preds, 'targets': targets})
        
        acc = (preds == targets).float().mean()
        self.log('val_loss', loss, prog_bar=True, batch_size=batch.num_graphs)
        self.log('val_acc', acc, prog_bar=True, batch_size=batch.num_graphs)
        return loss

    def on_validation_epoch_end(self):
        if not self.validation_step_outputs:
            return
            
        all_preds = torch.cat([x['preds'] for x in self.validation_step_outputs]).numpy()
        all_targets = torch.cat([x['targets'] for x in self.validation_step_outputs]).numpy()
        
        accuracy = accuracy_score(all_targets, all_preds)
        precision = precision_score(all_targets, all_preds, zero_division=0)
        recall = recall_score(all_targets, all_preds, zero_division=0)
        f1 = f1_score(all_targets, all_preds, zero_division=0)
        
        # IMPORTANT: Log these for the progress bar
        self.log('val_f1', f1, prog_bar=True)
        self.log('val_recall', recall, prog_bar=True)
        self.log('val_precision', precision)
        
        self.validation_step_outputs.clear()
        torch.cuda.empty_cache()

    def configure_optimizers(self):
        # Slower learning rate for Focal Loss stability
        optimizer = AdamW(self.parameters(), lr=3e-4, weight_decay=1e-5)
        
        # CRITICAL: Monitor val_f1 instead of val_loss
        scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)
        return {
            'optimizer': optimizer, 
            'lr_scheduler': {
                'scheduler': scheduler, 
                'monitor': 'val_f1' 
            }
        }

In [13]:
class TrackTransformer(torch.nn.Module):
    def __init__(self, in_channels=3, hidden_channels=32, out_channels=1, heads=4):
        super().__init__()
        self.hidden_channels = hidden_channels
        self.heads = heads
        
        self.embed = torch.nn.Linear(in_channels, hidden_channels)
        # FIX 7: Add BatchNorm for stability
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        
        self.conv1 = TransformerConv(hidden_channels, hidden_channels, heads=heads, 
                                     dropout=0.1, edge_dim=hidden_channels, beta=True)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels * heads)
        
        self.conv2 = TransformerConv(hidden_channels * heads, hidden_channels, heads=heads, 
                                     dropout=0.1, edge_dim=hidden_channels * heads, beta=True)
        self.bn3 = torch.nn.BatchNorm1d(hidden_channels * heads)
        
        self.conv3 = TransformerConv(hidden_channels * heads, hidden_channels // heads, heads=heads, 
                                     dropout=0.1, edge_dim=hidden_channels * heads, beta=True)
        
        self.lin_edge = torch.nn.Linear((hidden_channels // heads) * heads * 2, out_channels)

    def forward(self, x, edge_index, batch=None):
        x = self.embed(x)
        x = self.bn1(x)  # BatchNorm after embedding
        x = F.relu(x)
        
        # Layer 1
        src, tgt = x[edge_index[0]], x[edge_index[1]]
        edge_attr = torch.abs(src - tgt)
        x = self.conv1(x, edge_index, edge_attr)
        x = self.bn2(x)  # BatchNorm after conv
        x = F.relu(x)
        
        # Layer 2
        src, tgt = x[edge_index[0]], x[edge_index[1]]
        edge_attr = torch.abs(src - tgt)
        x = self.conv2(x, edge_index, edge_attr)
        x = self.bn3(x)  # BatchNorm after conv
        x = F.relu(x)
        
        # Layer 3
        src, tgt = x[edge_index[0]], x[edge_index[1]]
        edge_attr = torch.abs(src - tgt)
        x = self.conv3(x, edge_index, edge_attr)
        
        # Edge classification
        edge_emb = torch.cat([x[edge_index[0]], x[edge_index[1]]], dim=-1)
        return self.lin_edge(edge_emb).squeeze(-1)

In [14]:
# Prepare data with train/val split using multiple training files
root = '/kaggle/input/trackml-particle-identification'

In [15]:
# Load training data with more events (can handle more with batching!)
print("\nLoading training data...")
train_dataset = TrackMLDataset(root, zip_name='train_sample.zip', n_events=30)  # More events with batching!
print(f"  Training events: {len(train_dataset)}")


Loading training data...
  Training events: 30


In [16]:
# Load validation data
print("Loading validation data...")
val_dataset = TrackMLDataset(root, zip_name='train_sample.zip', n_events=10)
print(f"  Validation events: {len(val_dataset)}")

full_train_dataset = train_dataset

print(f"Dataset Summary:")
print(f"  Training:   {len(full_train_dataset)} events")
print(f"  Validation: {len(val_dataset)} events")



Loading validation data...
  Validation events: 10
Dataset Summary:
  Training:   30 events
  Validation: 10 events


In [17]:
BATCH_SIZE = 2  # Process 2 events at once

train_loader = DataLoader(
    full_train_dataset, 
    batch_size=BATCH_SIZE,  # Batch multiple graphs together!
    shuffle=True, 
    num_workers=3,
    pin_memory=True  # Faster GPU transfer
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE,  # Same batch size for validation
    shuffle=False, 
    num_workers=3,
    pin_memory=True
)

print(f" DataLoaders created:")
print(f"   Training batches: {len(train_loader)} (batch_size={BATCH_SIZE})")
print(f"   Validation batches: {len(val_loader)} (batch_size={BATCH_SIZE})")
print(f"   Total training graphs: {len(full_train_dataset)}")
print(f"   Total validation graphs: {len(val_dataset)}\n")

 DataLoaders created:
   Training batches: 15 (batch_size=2)
   Validation batches: 5 (batch_size=2)
   Total training graphs: 30
   Total validation graphs: 10



In [18]:
# Train
model = TrackTransformer()
lightning_model = TrackLightning(model)

In [19]:
import os
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# 1. Create directory safely
os.makedirs('checkpoints', exist_ok=True)

# 2. Re-define callbacks with clear monitoring
checkpoint_callback = ModelCheckpoint(
    monitor='val_f1',
    dirpath='checkpoints',
    filename='track-best-{epoch:02d}-{val_f1:.3f}',
    save_top_k=1,
    mode='max'
)

early_stop_callback = EarlyStopping(
    monitor='val_f1',
    patience=10,
    mode='max',
    verbose=True
)

# 3. Trainer with safety settings
trainer = pl.Trainer(
    max_epochs=25,
    accelerator='gpu',
    devices=1,
    precision='16-mixed',
    gradient_clip_val=0.5,
    callbacks=[checkpoint_callback, early_stop_callback],
    # If it still crashes, change this to 0 to skip the "practice" run
    num_sanity_val_steps=2 
)

print("🚀 Starting Training...")
trainer.fit(lightning_model, train_loader, val_loader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


🚀 Starting Training...


2026-01-04 12:18:12.740625: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767529092.955430      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767529093.016287      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767529093.525897      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767529093.525934      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767529093.525938      55 computation_placer.cc:177] computation placer alr

┏━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name    ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model   │ TrackTransformer │  125 K │ train │     0 │
│ 1 │ loss_fn │ FocalLoss        │      0 │ train │     0 │
└───┴─────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 125 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 125 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 31                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:317: The number of training batches 
(15) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if
you want to see logs for the training epoch.

Metric val_f1 improved. New best score: 0.039
Metric val_f1 improved by 0.003 >= min_delta = 0.0. New best score: 0.043
Monitored metric val_f1 did not improve in the last 10 records. Best score: 0.043. Signaling Trainer to stop.


In [21]:
# 1. Initialize and Load (Same as before)
base_model = TrackTransformer(in_channels=3, hidden_channels=32, out_channels=1, heads=2)
model_eval = TrackLightning.load_from_checkpoint(
    'checkpoints/trackml-epoch=04-val_f1=0.0700.ckpt', 
    model=base_model
)

# 2. Use validate() instead of test() to avoid the error
trainer.validate(model_eval, dataloaders=val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.8679221868515015     │
│       val_accuracy        │    0.8679222464561462     │
│          val_f1           │    0.06688123196363449    │
│         val_loss          │            nan            │
│       val_precision       │    0.03919358551502228    │
│        val_recall         │    0.22782272100448608    │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': nan,
  'val_acc': 0.8679221868515015,
  'val_accuracy': 0.8679222464561462,
  'val_precision': 0.03919358551502228,
  'val_recall': 0.22782272100448608,
  'val_f1': 0.06688123196363449}]